In [1]:
import pandas as pd
import os

def filter_existing_images(csv_path, image_folder, id_col, output_csv):
    df = pd.read_csv(csv_path)
    # Lọc những dòng mà file ảnh tồn tại
    df_filtered = df[df[id_col].apply(lambda x: os.path.isfile(os.path.join(image_folder, str(x))))]
    df_filtered.to_csv(output_csv, index=False)
    print(f"Đã lưu {len(df_filtered)} dòng vào {output_csv}")

# Ví dụ sử dụng:
# filter_existing_images('data.csv', 'images', 'id', 'filtered.csv')

In [4]:
filter_existing_images('./data/miccai2023_nih-cxr-lt_labels_test.csv', './data/images', 'id', './data/labels_test.csv')

Đã lưu 0 dòng vào ./data/labels_test.csv


In [3]:
filter_existing_images('./data/miccai2023_nih-cxr-lt_labels_train.csv', './data/images', 'id', './data/labels_train.csv')

Đã lưu 5000 dòng vào ./data/labels_train.csv


In [5]:
filter_existing_images('./data/miccai2023_nih-cxr-lt_labels_val.csv', './data/images', 'id', './data/labels_val.csv')

Đã lưu 0 dòng vào ./data/labels_val.csv


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Đọc dữ liệu gốc
df = pd.read_csv('./data/labels_train.csv')  # hoặc file tổng hợp đủ bản ghi

# Lọc các bản ghi theo Effusion
df_1 = df[df['Effusion'] == 1]
df_0 = df[df['Effusion'] == 0]

# Lấy số lượng phù hợp
n_1 = 5000 // 10           # ~1666 bản ghi Effusion=1
n_0 = 5000 - n_1          # ~3334 bản ghi Effusion=0

df_1_sample = df_1.sample(n=n_1, random_state=42)
df_0_sample = df_0.sample(n=n_0, random_state=42)

# Gộp lại và shuffle
df_all = pd.concat([df_1_sample, df_0_sample]).sample(frac=1, random_state=42).reset_index(drop=True)

# Chia train/val/test
train_df, temp_df = train_test_split(df_all, test_size=0.3, stratify=df_all['Effusion'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['Effusion'], random_state=42)

# Lưu file
train_df.to_csv('./data/train_effusion.csv', index=False)
val_df.to_csv('./data/val_effusion.csv', index=False)
test_df.to_csv('./data/test_effusion.csv', index=False)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
print("Effusion distribution in train:", train_df['Effusion'].value_counts())
print("Effusion distribution in val:", val_df['Effusion'].value_counts())
print("Effusion distribution in test:", test_df['Effusion'].value_counts())

Train: 3500, Val: 750, Test: 750
Effusion distribution in train: Effusion
0    2334
1    1166
Name: count, dtype: int64
Effusion distribution in val: Effusion
0    500
1    250
Name: count, dtype: int64
Effusion distribution in test: Effusion
0    500
1    250
Name: count, dtype: int64


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Đọc dữ liệu gốc
df = pd.read_csv('./data/labels_train.csv')  # hoặc file tổng hợp đủ bản ghi

# Lọc các bản ghi theo Effusion
df_1 = df[df['Effusion'] == 1]
df_0 = df[df['Effusion'] == 0]

# Lấy số lượng phù hợp
n_1 = 5000 // 3           # 1666 bản ghi Effusion=1
n_0 = 5000 - n_1          # 3334 bản ghi Effusion=0

df_1_sample = df_1.sample(n=n_1, random_state=42)
df_0_sample = df_0.sample(n=n_0, random_state=42)

# Gộp lại và shuffle
df_all = pd.concat([df_1_sample, df_0_sample]).sample(frac=1, random_state=42).reset_index(drop=True)

# Chia train/val/test theo tỷ lệ 70/15/15, giữ nguyên tỷ lệ nhãn
train_df, temp_df = train_test_split(df_all, test_size=0.3, random_state=42, shuffle=True)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, shuffle=True)

# Kiểm tra lại tỷ lệ
print("Train Effusion=1:", train_df['Effusion'].mean())
print("Val Effusion=1:", val_df['Effusion'].mean())
print("Test Effusion=1:", test_df['Effusion'].mean())

# Lưu file
train_df.to_csv('./data/train_effusion.csv', index=False)
val_df.to_csv('./data/val_effusion.csv', index=False)
test_df.to_csv('./data/test_effusion.csv', index=False)

Train Effusion=1: 0.33714285714285713
Val Effusion=1: 0.34
Test Effusion=1: 0.308


In [ ]:
print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
print("Effusion distribution in train:", train_df['Effusion'].value_counts())
print("Effusion distribution in val:", val_df['Effusion'].value_counts())
print("Effusion distribution in test:", test_df['Effusion'].value_counts())

Train: 3500, Val: 750, Test: 750
Effusion distribution in train: Effusion
0    2320
1    1180
Name: count, dtype: int64
Effusion distribution in val: Effusion
0    495
1    255
Name: count, dtype: int64
Effusion distribution in test: Effusion
0    519
1    231
Name: count, dtype: int64


: 

In [9]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# 1. Load model pretrained hoặc custom
model = models.resnet18(pretrained=True)
model.eval()

# 2. Load ảnh
image_path = 'xray_image.png'  # Đường dẫn ảnh X-quang
image = Image.open(image_path).convert('RGB')

# 3. Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # chuẩn cho ImageNet
                         std=[0.229, 0.224, 0.225])
])

input_tensor = transform(image).unsqueeze(0)  # Thêm batch dim

# 4. Lấy layer cuối cùng trước fc (tùy theo model bạn dùng)
target_layer = model.layer4[-1]

# 5. Sử dụng Grad-CAM
cam = GradCAM(model=model, target_layers=[target_layer], use_cuda=False)

# Nếu bạn muốn focus vào 1 class cụ thể (ví dụ: class 1)
targets = None  # hoặc [ClassifierOutputTarget(class_idx)]

grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0]

# 6. Visualize Grad-CAM
rgb_img = cv2.cvtColor(np.array(image.resize((224, 224))), cv2.COLOR_RGB2BGR)
visualization = visualize_cam(grayscale_cam, rgb_img)

plt.imshow(visualization)
plt.title("Grad-CAM - Vùng mô hình tập trung")
plt.axis('off')
plt.show()


ModuleNotFoundError: No module named 'pytorch_grad_cam'

In [8]:
!pip list

Package                           Version
--------------------------------- ------------------
aiobotocore                       2.12.3
aiohttp                           3.9.5
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.16
altair                            5.0.1
anaconda-anon-usage               0.4.4
anaconda-catalogs                 0.2.0
anaconda-client                   1.12.3
anaconda-cloud-auth               0.5.1
anaconda-navigator                2.6.2
anaconda-project                  0.11.1
annotated-types                   0.6.0
anyio                             4.2.0
appdirs                           1.4.4
archspec                          0.2.3
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.3
astroid                           2.14.2
astropy                           6.1.0
astropy-iers-data                 0.2024.6.3.0.31.14
astto

In [1]:
import os
import pandas as pd

# Đọc danh sách id từ 3 file csv
train_ids = pd.read_csv('./data/train_effusion.csv')['id'].astype(str)
val_ids = pd.read_csv('./data/val_effusion.csv')['id'].astype(str)
test_ids = pd.read_csv('./data/test_effusion.csv')['id'].astype(str)

used_ids = set(train_ids) | set(val_ids) | set(test_ids)

image_folder = './data/images'

# Liệt kê tất cả file trong thư mục ảnh
for fname in os.listdir(image_folder):
    if fname not in used_ids:
        try:
            os.remove(os.path.join(image_folder, fname))
            print(f"Đã xóa: {fname}")
        except Exception as e:
            print(f"Lỗi khi xóa {fname}: {e}")

Đã xóa: 00000001_001.png
Đã xóa: 00000002_000.png
Đã xóa: 00000003_000.png
Đã xóa: 00000003_001.png
Đã xóa: 00000003_002.png
Đã xóa: 00000003_003.png
Đã xóa: 00000003_004.png
Đã xóa: 00000003_005.png
Đã xóa: 00000003_006.png
Đã xóa: 00000003_007.png
Đã xóa: 00000005_000.png
Đã xóa: 00000005_001.png
Đã xóa: 00000005_003.png
Đã xóa: 00000005_004.png
Đã xóa: 00000005_005.png
Đã xóa: 00000005_006.png
Đã xóa: 00000005_007.png
Đã xóa: 00000006_000.png
Đã xóa: 00000007_000.png
Đã xóa: 00000008_000.png
Đã xóa: 00000008_001.png
Đã xóa: 00000008_002.png
Đã xóa: 00000009_000.png
Đã xóa: 00000010_000.png
Đã xóa: 00000011_002.png
Đã xóa: 00000011_004.png
Đã xóa: 00000011_005.png
Đã xóa: 00000011_006.png
Đã xóa: 00000011_007.png
Đã xóa: 00000011_008.png
Đã xóa: 00000012_000.png
Đã xóa: 00000013_000.png
Đã xóa: 00000013_001.png
Đã xóa: 00000013_002.png
Đã xóa: 00000013_003.png
Đã xóa: 00000013_004.png
Đã xóa: 00000013_005.png
Đã xóa: 00000013_006.png
Đã xóa: 00000013_007.png
Đã xóa: 00000013_008.png
